In [5]:
import pandas as pd
from datetime import datetime
import numpy as np
import json

In [6]:
dataEvents = pd.read_csv('/Users/manuelgomezmoratilla/Desktop/TFG/data_processing_scripts/data/anonamyze_all_data_collection_v2.csv', sep=";")

In [7]:
def computePlayStyles(dataEvents,  puzzles = 'all', group = 'all'):
    if puzzles == 'all':
        puzzles = ['Square Cross-Sections', 'Bird Fez', 'Pi Henge', '45-Degree Rotations',  'Pyramids are Strange', 'Boxes Obscure Spheres', 'Object Limits', 'Angled Silhouette',
                  'Sugar Cones','Stranger Shapes', 'Tall and Small', 'Ramp Up and Can It', 'More Than Meets Your Eye', 'Not Bird', 'Zzz', 'Bull Market', 'Orange Dance', 'Bear Market']

    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')
    
    #iterates in the groups and users of the data
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['task_id'] = [json.loads(x)['task_id'] if 'task_id' in json.loads(x).keys() else '' for x in dataEvents['data']]
    
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    dataEvents['group_user_task_id'] = dataEvents['group'] + '~' + dataEvents['user']+'~'+dataEvents['task_id']

         
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    
    newEvents = []
    for user in dataEvents['group_user_id'].unique():
        user_events = dataEvents[dataEvents['group_user_id'] == user]
        user_events_na_dropped = user_events.dropna()
        activePuzzle = None
        nAttempt = 1
        for enum, event in user_events_na_dropped.iterrows():
            if(event['type'] == 'ws-start_level'):
                activePuzzle = json.loads(event['data'])['task_id']
                if activePuzzle == None :
                    continue
                event['task_id'] = activePuzzle
                event['n_attempt'] = nAttempt
                event['group_user_task_id'] = event['group'] + '~' + event['user'] + '~' + event['task_id'] + '~' + str(nAttempt)
                if (event['task_id'] == 'Sandbox'):
                    continue
                newEvents.append(event)
            elif(event['type'] in ['ws-exit_to_menu', 'ws-disconnect', 'ws-login_user']):
                if (activePuzzle == 'Sandbox'):
                    continue
                event['task_id'] = activePuzzle
                if activePuzzle == None :
                    continue
                event['n_attempt'] = nAttempt
                event['group_user_task_id'] = event['group'] + '~' + event['user'] + '~' + event['task_id'] + '~' + str(nAttempt)
                nAttempt +=1
                newEvents.append(event)
            else:
                if (activePuzzle == 'Sandbox'):
                    continue
                event['task_id'] = activePuzzle
                if activePuzzle == None :
                    continue
                event['n_attempt'] = nAttempt
                event['group_user_task_id'] = event['group'] + '~' + event['user'] + '~' + event['task_id'] + '~' + str(nAttempt)
                newEvents.append(event)
                
    initialDf = pd.DataFrame(newEvents, columns=['id', 'time', 'group_user_id', 'group', 'user', 'group_user_task_id', 'n_attempt', 'task_id', 'type', 'data'])
    #Recalculate n_attempt
    mod = []
    for user in initialDf['user'].unique():
            previousAttempt = 1
            n_attempt = 1
            individualDf = initialDf[initialDf['user'] == user]
            for enum, event in individualDf.iterrows():
                if (event['n_attempt'] != previousAttempt):
                    n_attempt += 1
                previousAttempt = event['n_attempt']
                event['n_attempt'] = n_attempt
                mod.append(event)
    modDf = pd.DataFrame(mod, columns=['id', 'time', 'group_user_id', 'group', 'user', 'group_user_task_id', 'n_attempt', 'task_id', 'type', 'data'])
    
    initialDf = modDf
    
    data_car = []
    for puzzle in puzzles:
        initialDfP = initialDf[initialDf['task_id'] == puzzle]
        # the data is grouped by the necessary variables      
        activity_by_user = initialDfP.groupby(['group_user_id','group', 'user','group_user_task_id','task_id', 'n_attempt']).agg({'id':'count',
                                                 'type':'nunique'}).reset_index().rename(columns={'id':'events',
                                                                                                  'type':'different_events'}) 

        #indicate the index variable                                                                                                                                                               
        activity_by_user.index = activity_by_user['group_user_task_id'].values

        #initialize the metrics                                                                                          
        activity_by_user['event'] = np.nan
        activity_by_user['different_events'] = np.nan
        activity_by_user['active_time'] = np.nan
        activity_by_user['snapshot'] = 0.0
        activity_by_user['paint'] = 0
        activity_by_user['rotate_view'] = 0.0
        activity_by_user['move_shape'] = 0
        activity_by_user['scale_shape'] = 0
        activity_by_user['create_shape'] = 0
        activity_by_user['delete_shape'] = 0
        activity_by_user['undo_action'] = 0
        activity_by_user['redo_action'] = 0
        activity_by_user['restart_puzzle'] = 0
        activity_by_user['submit_action'] = 0.0
        activity_by_user['manipulation_events'] = 0.0
        activity_by_user['firstShape_time'] = 0.0

        #initialize the data structures
        userFunnelDict = dict()  
        puzzleEvents = dict()
        eventsDiff = []
        eventsDiff_puzzle = dict()
        timePuzzle = dict()
        globalTypesEvents = dict()
        typesEvents = dict()


        for user in initialDfP['group_user_id'].unique():

            # Computing active time
            previousEvent = None
            theresHoldActivity = 60 # np.percentile(allDifferences, 98) is 10 seconds
            activeTime = []

            user_events = initialDfP[initialDfP['group_user_id'] == user]
            user_puzzle_key = None

            for enum, event in user_events.iterrows():

                # If it is the first event
                if(previousEvent is None):
                    previousEvent = event
                    continue

                if(event['type'] in ['ws-start_level', 'ws-puzzle_started']):

                    #create id: group+user+task_id                                                                              
                    user_puzzle_key =  event['group_user_task_id']
                    firstEvent = event
                    shapeCreated = False

                    # initialize if the id is new                                                                              
                    if(user_puzzle_key not in puzzleEvents.keys()):
                        puzzleEvents[user_puzzle_key]= 1
                        eventsDiff_puzzle[user_puzzle_key] = []
                        timePuzzle[user_puzzle_key] = 0

                        globalTypesEvents[user_puzzle_key] = dict()
                        globalTypesEvents[user_puzzle_key]['ws-snapshot'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-paint'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-rotate_view'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-move_shape'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-scale_shape'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-create_shape'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-delete_shape'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-undo_action'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-redo_action'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-check_solution'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-man_events'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-restart_puzzle'] = 0
                        globalTypesEvents[user_puzzle_key]['ws-firstShape_time'] = 0.0
                        globalTypesEvents[user_puzzle_key]['ws-puzzle_complete'] = False


                        eventsDiff_puzzle[user_puzzle_key].append(event['type'])

                # the event is not final event
                if(event['type'] not in ['ws-exit_to_menu', 'ws-create_user', 'ws-login_user']): 

                        puzzleEvents[user_puzzle_key] += 1

                        #add the event type                                                                          
                        eventsDiff_puzzle[user_puzzle_key].append(event['type'])

                        #calculate the duration of the event                                                                          
                        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                        if((delta_seconds < theresHoldActivity)):
                            timePuzzle[user_puzzle_key] += delta_seconds

                        previousEvent = event 

                        #update event counters by type                                                                          
                        if(event['type'] == 'ws-snapshot'):
                            globalTypesEvents[user_puzzle_key]['ws-snapshot'] +=1
                            globalTypesEvents[user_puzzle_key]['ws-man_events'] +=1
                        elif(event['type'] == 'ws-rotate_view'):
                            globalTypesEvents[user_puzzle_key]['ws-rotate_view'] +=1 
                        elif(event['type'] == 'ws-paint'):
                            globalTypesEvents[user_puzzle_key]['ws-paint'] +=1 
                        elif(event['type'] == 'ws-move_shape'):
                            globalTypesEvents[user_puzzle_key]['ws-move_shape'] +=1 
                            globalTypesEvents[user_puzzle_key]['ws-man_events'] +=1
                        elif(event['type'] == 'ws-scale_shape'):
                            globalTypesEvents[user_puzzle_key]['ws-scale_shape'] +=1
                            globalTypesEvents[user_puzzle_key]['ws-man_events'] +=1
                        elif(event['type'] == 'ws-create_shape'):
                            globalTypesEvents[user_puzzle_key]['ws-create_shape'] +=1
                            globalTypesEvents[user_puzzle_key]['ws-man_events'] +=1
                            if shapeCreated == False:
                                shapeCreated = True
                                delta_shape = (event['time'] - firstEvent['time']).total_seconds()
                                globalTypesEvents[user_puzzle_key]['ws-firstShape_time'] = delta_shape
                        elif(event['type'] == 'ws-delete_shape'):
                            globalTypesEvents[user_puzzle_key]['ws-delete_shape'] +=1
                            globalTypesEvents[user_puzzle_key]['ws-man_events'] +=1
                        elif(event['type'] == 'ws-undo_action'):
                            globalTypesEvents[user_puzzle_key]['ws-undo_action'] +=1
                        elif(event['type'] == 'ws-redo_action'):
                            globalTypesEvents[user_puzzle_key]['ws-redo_action'] +=1 
                        elif(event['type'] == 'ws-check_solution'):
                            globalTypesEvents[user_puzzle_key]['ws-check_solution'] +=1 
                        elif(event['type'] == 'ws-restart_puzzle'):
                            globalTypesEvents[user_puzzle_key]['ws-restart_puzzle'] +=1 
                        elif(event['type'] == 'ws-puzzle_complete'):
                            globalTypesEvents[user_puzzle_key]['ws-puzzle_complete'] = True


                # the puzzle ends        
                if(event['type'] in ['ws-exit_to_menu', 'ws-puzzle_complete']):

                        #add the event type                                                                         
                        eventsDiff_puzzle[user_puzzle_key].append(event['type'])

                        #calculate the duration of the event                                                                          
                        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                        if((delta_seconds < theresHoldActivity)):
                            timePuzzle[user_puzzle_key] += delta_seconds

                        previousEvent = event

        # add the data by group_user_task_id 
        for i in activity_by_user['group_user_task_id'].unique():
            key_split = i.split('~')
            if(key_split[2] != ''):
                if puzzleEvents.get(i) != None:
                    activity_by_user.at[i, 'event'] = puzzleEvents[i]
                    activity_by_user.at[i, 'different_events'] = len(set(eventsDiff_puzzle[i]))
                    activity_by_user.at[i, 'active_time'] = timePuzzle[i]
                    if (globalTypesEvents[i]['ws-man_events'] > 0):
                        activity_by_user.at[i, 'snapshot'] = globalTypesEvents[i]['ws-snapshot'] / globalTypesEvents[i]['ws-man_events']
                    else:
                        activity_by_user.at[i, 'snapshot'] = 0
                    activity_by_user.at[i, 'paint'] = globalTypesEvents[i]['ws-paint']
                    if (globalTypesEvents[i]['ws-man_events'] + globalTypesEvents[i]['ws-rotate_view']) > 0 :
                        activity_by_user.at[i, 'rotate_view'] = globalTypesEvents[i]['ws-rotate_view'] / (globalTypesEvents[i]['ws-man_events'] + globalTypesEvents[i]['ws-rotate_view'])
                    else:
                        activity_by_user.at[i, 'rotate_view'] = 0
                    activity_by_user.at[i, 'move_shape'] = globalTypesEvents[i]['ws-move_shape']
                    activity_by_user.at[i, 'scale_shape'] = globalTypesEvents[i]['ws-scale_shape']
                    activity_by_user.at[i, 'create_shape'] = globalTypesEvents[i]['ws-create_shape']
                    activity_by_user.at[i, 'delete_shape'] = globalTypesEvents[i]['ws-delete_shape']
                    activity_by_user.at[i, 'undo_action'] = globalTypesEvents[i]['ws-undo_action']
                    activity_by_user.at[i, 'redo_action'] = globalTypesEvents[i]['ws-redo_action']
                    activity_by_user.at[i, 'restart_puzzle'] = globalTypesEvents[i]['ws-restart_puzzle']
                    if (globalTypesEvents[i]['ws-man_events'] + globalTypesEvents[i]['ws-check_solution']) > 0:
                        activity_by_user.at[i, 'submit_events'] = globalTypesEvents[i]['ws-check_solution'] / (globalTypesEvents[i]['ws-man_events'] + globalTypesEvents[i]['ws-check_solution'])
                    else:
                        activity_by_user.at[i, 'submit_events'] = 0
                    activity_by_user.at[i, 'completed'] = globalTypesEvents[i]['ws-puzzle_complete']
                    if activity_by_user.at[i, 'active_time'] > 0 :
                        manByTime = globalTypesEvents[i]['ws-man_events'] / activity_by_user.at[i, 'active_time']
                    else:
                        manByTime = globalTypesEvents[i]['ws-man_events']
                    activity_by_user.at[i, 'manipulation_events'] = manByTime
                    activity_by_user.at[i, 'firstShape_time'] = globalTypesEvents[i]['ws-firstShape_time']
                    if activity_by_user.at[i, 'submit_events'] > 0 :
                        activity_by_user.at[i, 'eventsBySubmit'] =  activity_by_user.at[i, 'event'] / globalTypesEvents[i]['ws-check_solution']
                    else:
                        activity_by_user.at[i, 'eventsBySubmit'] =  0.0


        #delete row with NaN
        activity_by_user.dropna(inplace=True)
        #delete group_user_task_id column
        activity_by_user.drop(columns=['group_user_task_id'], inplace=True)

        #data output preparation                                                                                          
        activity_by_user = pd.melt(activity_by_user, id_vars=['group', 'user','task_id', 'n_attempt'], 
            value_vars=['event','different_events', 'active_time','snapshot','paint','rotate_view','move_shape','scale_shape','create_shape','delete_shape','undo_action','redo_action', 'submit_events', 'firstShape_time', 'manipulation_events', 'eventsBySubmit', 'completed', 'restart_puzzle'], 
            var_name='metric', value_name='value')

        df2 = activity_by_user
        for user in df2['user'].unique():
            data_user = df2[df2['user'] == user]
            for attempt in data_user['n_attempt'].unique():
                attempt_data = data_user[data_user['n_attempt'] == attempt]
                for metric in attempt_data['metric'].unique():
                    metric_data = attempt_data[attempt_data['metric'] == metric]
                    for enum, event in metric_data.iterrows():
                        event['unique_id'] = event['group'] + '~' + event['user'] + '~' +  event['task_id'] + '~' + str(event['n_attempt'])
                        data_car.append(event)
                        
    modDf = pd.DataFrame(data_car, columns=['group', 'unique_id', 'metric' ,'value'])
        
        #Define all dimensions
    spatialDimension = ['snapshot', 'rotate_view']
    experimentDimension = ['submit_events', 'manipulation_events']
    reflexDimension = ['firstShape_time', 'eventsBySubmit']
    exploreDimension = ['different_events', 'paint']
    engageDimension = ['active_time', 'event']
    errorRectDimension = ['delete_shape', 'restart_puzzle']
    dimensions = [spatialDimension, experimentDimension, reflexDimension, exploreDimension, engageDimension, errorRectDimension]

    df3 = modDf
    df = df3.pivot(index='unique_id', columns='metric', values='value')
    newEvents = []
    for enum, event in df.iterrows():
        ev = event.copy()
        for dim in dimensions:
            ev[dim] = 0.0
            total = 0.0
            for feature in dim:
                total += event[feature]
            if dim == spatialDimension :
                ev['spatialDimension'] = total
            elif dim == experimentDimension:
                ev['experimentDimension'] = total
            elif dim == reflexDimension:
                ev['reflexDimension'] = total
            elif dim == exploreDimension:
                ev['exploreDimension'] = total
            elif dim == errorRectDimension:
                ev['errorRectDimension'] = total
            else:
                ev['engageDimension'] = total
        newEvents.append(ev)

    new = pd.DataFrame(newEvents, columns = ['spatialDimension', 'experimentDimension', 'exploreDimension', 'reflexDimension', 'engageDimension', 'errorRectDimension', 'completed'])
  
    that = []
    for ind in new.index.values:
        ob = new.loc[ind,:].copy()
        key_split = ind.split('~')
        ob['group_id'] = key_split[0]
        ob['user'] = key_split[1]
        ob['task_id'] = key_split[2]
        ob['n_attempt'] = key_split[3]
        that.append(ob)
    thatDf = pd.DataFrame(that, columns=['group_id', 'user', 'task_id', 'n_attempt','spatialDimension', 'experimentDimension', 'exploreDimension', 'reflexDimension', 'engageDimension', 'errorRectDimension', 'completed']) 
    thatDf.reset_index(drop=True, inplace=True)
    
    finalList = []
    for puzzle in thatDf['task_id'].unique():
        newPuzzle = thatDf[thatDf['task_id'] == puzzle]
        percSpatialUp = newPuzzle['spatialDimension'].quantile(q=0.75)
        percSpatialDown = newPuzzle['spatialDimension'].quantile(q=0.40)

        percExploreUp = newPuzzle['exploreDimension'].quantile(q=0.75)
        percExploreDown = newPuzzle['exploreDimension'].quantile(q=0.20)

        percEngageUp = newPuzzle['engageDimension'].quantile(q=0.75)
        percEngageDown = newPuzzle['engageDimension'].quantile(q=0.20)

        percExperimentUp = newPuzzle['experimentDimension'].quantile(q=0.75)
        percExperimentDown = newPuzzle['experimentDimension'].quantile(q=0.20)

        percReflexUp = newPuzzle['reflexDimension'].quantile(q=0.75)
        percReflexDown = newPuzzle['reflexDimension'].quantile(q=0.20)

        percErrorRectUp = newPuzzle['errorRectDimension'].quantile(q=0.75)
        percErrorRectDown = newPuzzle['errorRectDimension'].quantile(q=0.20)

        for enum, event in newPuzzle.iterrows():

            event['percSpatialUp'] = percSpatialUp
            event['percSpatialDown'] = percSpatialDown

            event['percEngageUp'] = percEngageUp
            event['percEngageDown'] = percEngageDown

            event['percExperimentUp'] = percExperimentUp
            event['percExperimentDown'] = percExperimentDown

            event['percExploreUp'] = percExploreUp
            event['percExploreDown'] = percExploreDown

            event['percReflexUp'] = percReflexUp
            event['percReflexDown'] = percReflexDown

            event['percErrorRectUp'] = percErrorRectUp
            event['percErrorRectDown'] = percErrorRectDown

            if event['spatialDimension'] > percSpatialUp:
                event['ConcreteThinker'] = True
                event['AbstractThinker'] = False
            elif event['spatialDimension'] < percSpatialDown:
                event['ConcreteThinker'] = False
                event['AbstractThinker'] = True
            else:
                event['ConcreteThinker'] = False
                event['AbstractThinker'] = False

            if event['exploreDimension'] > percExploreUp:
                event['Explorer'] =  True
                event['Non-Explorer'] = False
            elif event['exploreDimension'] < percExploreDown:
                event['Non-Explorer'] = True
                event['Explorer'] = False
            else:
                event['Non-Explorer'] = False
                event['Explorer'] = False

            if event['experimentDimension'] > percExperimentUp:
                event['Experimenter'] = True
                event['Non-Experimenter'] = False
            elif event['experimentDimension'] < percExperimentDown:
                event['Experimenter'] = False
                event['Non-Experimenter'] = True
            else:
                event['Experimenter'] = False
                event['Non-Experimenter'] = False

            if event['engageDimension'] > percEngageUp:
                event['UnproductiveStruggle'] = True
                event['RapidSolver'] = False
            elif event['engageDimension'] < percEngageDown and event['completed'] == True:
                event['RapidSolver'] = True
                event['UnproductiveStruggle'] =  False
            else:
                event['UnproductiveStruggle'] = False
                event['RapidSolver'] = False

            if event['reflexDimension'] > percReflexUp:
                event['HighlyReflexive'] = True
                event['Non-Reflexive'] = False
            elif event['reflexDimension'] < percReflexDown:
                event['HighlyReflexive'] = False
                event['Non-Reflexive'] = True
            else:
                event['HighlyReflexive'] = False
                event['Non-Reflexive'] = False

            if event['errorRectDimension'] > percErrorRectUp:
                event['IveGotThis'] = False
                event['CleanSlate'] = True
            elif event['errorRectDimension'] < percErrorRectDown and event['completed'] == True:
                event['IveGotThis'] = True
                event['CleanSlate'] = False
            else:
                event['CleanSlate'] = False
                event['IveGotThis'] = False

            finalList.append(event)

    finalDf = pd.DataFrame(finalList, columns=['group_id', 'user', 'task_id', 'n_attempt', 'spatialDimension', 'percSpatialUp', 'percSpatialDown', 'ConcreteThinker', 'AbstractThinker', 'engageDimension', 'percEngageUp', 'percEngageDown', 'UnproductiveStruggle', 'RapidSolver', 'exploreDimension', 'percExploreUp', 'percExploreDown', 'Explorer', 'Non-Explorer', 'experimentDimension', 'percExperimentUp', 'percExperimentDown', 'Experimenter', 'Non-Experimenter', 'reflexDimension', 'percReflexUp', 'percReflexDown', 'HighlyReflexive', 'Non-Reflexive', 'errorRectDimension', 'percErrorRectUp', 'percErrorRectDown', 'IveGotThis', 'CleanSlate'])
    return finalDf

In [8]:
df2 = computePlayStyles(dataEvents)

In [9]:
df2

,group_id,user,task_id,n_attempt,spatialDimension,percSpatialUp,percSpatialDown,ConcreteThinker,AbstractThinker,engageDimension,...,reflexDimension,percReflexUp,percReflexDown,HighlyReflexive,Non-Reflexive,errorRectDimension,percErrorRectUp,percErrorRectDown,IveGotThis,CleanSlate
0,4fe25833f555e9903d2bb6bbeec3fbfb,12e1e72e84bf81d1bb70ceaaff147ee1,Square Cross-Sections,10,0.272501,0.505052,0.162162,False,False,409.550053,...,44.635019,59.604291,19.042475,False,False,5.0,3.0,0.0,False,True
7,4fe25833f555e9903d2bb6bbeec3fbfb,2085341eede3ffb7e2ffc72bbf0ae02b,Square Cross-Sections,10,0.142857,0.505052,0.162162,False,True,130.909455,...,70.517624,59.604291,19.042475,True,False,0.0,3.0,0.0,False,False
9,4fe25833f555e9903d2bb6bbeec3fbfb,2b2396af4cbf91224f099e909c6068f7,Square Cross-Sections,11,0.830526,0.505052,0.162162,True,False,461.974698,...,35.822438,59.604291,19.042475,False,False,3.0,3.0,0.0,False,False
11,4fe25833f555e9903d2bb6bbeec3fbfb,516f1d9200914db303f76bbefbeff1ad,Square Cross-Sections,10,0.475244,0.505052,0.162162,False,False,733.977151,...,57.223288,59.604291,19.042475,False,False,3.0,3.0,0.0,False,False
19,4fe25833f555e9903d2bb6bbeec3fbfb,66e12afc561f043a0ff56910f2b166d0,Square Cross-Sections,18,0.000000,0.505052,0.162162,False,True,607.718147,...,61.349121,59.604291,19.042475,True,False,0.0,3.0,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2871,e6af7d42084352a39449e6d0a09b18cd,f22d5e004366e83c9eb106b13b21bc9d,Tall and Small,23,0.424242,0.436147,0.161400,False,False,107.120680,...,48.572566,45.035510,13.359378,True,False,0.0,1.0,0.0,False,False
2890,e6af7d42084352a39449e6d0a09b18cd,f50c8d5d6414f0e64ef8c21973d0a6d6,Tall and Small,22,0.629789,0.436147,0.161400,True,False,658.517819,...,24.996787,45.035510,13.359378,False,False,1.0,1.0,0.0,False,False
2913,e6af7d42084352a39449e6d0a09b18cd,f520f6763fdb355659089797cfbd25e0,Tall and Small,22,0.350000,0.436147,0.161400,False,False,526.474069,...,19.639914,45.035510,13.359378,False,False,1.0,1.0,0.0,False,False
2957,e6af7d42084352a39449e6d0a09b18cd,fb039fa921dd55d1b7eb210d8a5f75fc,Tall and Small,21,0.000000,0.436147,0.161400,False,True,102.801466,...,34.505279,45.035510,13.359378,False,False,0.0,1.0,0.0,False,False


In [10]:
df2.to_csv("PlayStylesOutput.csv", decimal = ".")